In [32]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tomorrow import threads
from datetime import datetime
import time
import re
import random
import pandas as pd

In [33]:
ua = UserAgent()
headers = {'User-Agent': ua.random}
# 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'

@threads(10)
def Multithreads(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

In [36]:
links = []

for i in range(1, 34, 1):
    url = 'https://search.ltn.com.tw/list?keyword=%E8%90%8A%E8%B1%AC&start_time=20041201&end_time=20220103&sort=date&type=all&page={}'.format(i)
    # rawdata = requests.get(url, headers = headers)
    rawdata = Multithreads(url)
    time.sleep(random.uniform(1, 10))
    soup = BeautifulSoup(rawdata.text, 'html.parser')
    linkage = soup.find_all(class_ = "cont")
    for nodes in linkage:
        link = nodes.get('href')
        links.append(link)

# https://search.ltn.com.tw/list?keyword=%E5%8F%8D%E8%90%8A%E8%B1%AC&start_time=20041201&end_time=20211207&sort=date&type=all&page=2

In [37]:
pork = []


for link in links:
    if 'politics' in link:
        pork.append(link)
    elif 'talk.ltn.com.tw/article/paper' in link:
        pork.append(link)
    elif 'talk.ltn.com.tw/article/breakingnews' in link:
        pork.append(link)
print(len(links), len(pork))

660 547


In [23]:
# https://talk.ltn.com.tw/article/paper/1488324
# https://talk.ltn.com.tw/article/paper/1488138

# response = Multithreads("https://news.ltn.com.tw/news/life/breakingnews/3786612")
response = Multithreads("https://talk.ltn.com.tw/article/breakingnews/3748017")
# response = Multithreads("https://talk.ltn.com.tw/article/paper/1488138")
# response = Multithreads("https://ec.ltn.com.tw/article/breakingnews/3747024")

soup = BeautifulSoup(response.text, 'html.parser')
title = soup.find("h1").text
date = datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'], '%Y-%m-%dT%H:%M:%S+08:00')
content = ""

# soup.find_all("p", class_ = None)

soup.find("div", attrs = {'class':'whitecon'}).select("p:not([class]):not(:has(*))")[:-14]
print(title, date)
# soup.select("p:not([class]):not(:has(*))")
# soup.select("p:not([class]):not(:has(*))")[3:-10]
# if len(soup.select("p:not([class]):not(:has(*))")) > 4: # 濾掉影片

自由開講》日韓元對台幣大貶看台美貿易協定的重要性 2021-11-25 13:08:15


In [38]:
all_posts = []

for link in pork:
    if 'politics' in link:
        try:
            response = Multithreads(link)
            time.sleep(random.uniform(10, 20))
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.find("h1").text
            date = datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'], '%Y-%m-%dT%H:%M:%S+08:00')
            content = ""        
            if len(soup.select("p:not([class]):not(:has(*))")) > 4:
                for p in soup.select("p:not([class]):not(:has(*))")[3:-1]:
                    content += p.getText().strip()
                all_posts.append({'title':title, 
                                  'date':date, 
                                  'link':link, 
                                  'content':content})
        except:
            pass
    
    if 'talk.ltn.com.tw/article/paper' in link:
        try:
            response = Multithreads(link)
            time.sleep(random.uniform(10, 20))
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.find("h1").text
            date = datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'], '%Y-%m-%dT%H:%M:%S+08:00')
            content = ""
            if len(soup.select("p:not([class]):not(:has(*))")) > 4:
                for p in soup.select("p:not([class]):not(:has(*))")[3:-10]:
                    content += p.getText().strip()
                all_posts.append({'title':title, 
                                  'date':date, 
                                  'link':link, 
                                  'content':content})
        except:
            pass
    
    if 'talk.ltn.com.tw/article/breakingnews' in link:
        try:
            response = Multithreads(link)
            time.sleep(random.uniform(10, 20))
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.find("h1").text
            date = datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'], '%Y-%m-%dT%H:%M:%S+08:00')
            content = ""
            if len(soup.select("p:not([class]):not(:has(*))")) > 4:
                for p in soup.select("p:not([class]):not(:has(*))")[2:-16]:
                    content += p.getText().strip()
                all_posts.append({'title':title, 
                                  'date':date, 
                                  'link':link, 
                                  'content':content})
        except:
            pass

In [75]:
ltn_politics = pd.DataFrame(all_posts)
ltn_politics = ltn_politics.drop_duplicates(subset = ['link']).reset_index(drop = True)
# ltn_politics = ltn_politics.sort_values('date', ascending = False).reset_index(drop = True)
ltn_politics

,title,date,link,content
0,總統今發表2022新年談話 「穩健執政 讓國家進步前進」,2022-01-01 05:30:00,https://news.ltn.com.tw/news/politics/paper/14...,蔡英文總統今天將在總統府大禮堂發表「二○二二新年談話」，向國人傳達今年的重要施政方針。（資料...
1,自由廣場》唉，這樣的楊志良……,2021-12-31 05:30:00,https://talk.ltn.com.tw/article/paper/1493110,曾任馬政府衛生署長的楊志良日前在一場群眾活動裡，除了用「假論文」的莫須有罪名扣在蔡英文總統身...
2,謝長廷請假缺席立院報告 陳以信不滿,2021-12-30 17:00:22,https://news.ltn.com.tw/news/politics/breaking...,立法院外交及國防委員會召委、國民黨立委陳以信批評謝長廷不願面對國會。（記者楊丞彧攝）〔記者楊...
3,朱酸：沒人在乎林智堅是否要選,2021-12-30 05:30:00,https://news.ltn.com.tw/news/politics/paper/14...,朱立倫昨反酸，沒人在乎林智堅是否要選大新竹。（資料照）〔記者楊媛婷／台北報導〕新竹市長林智堅...
4,酸沒人在乎林智堅選不選 朱立倫：歡迎蔡總統討論國土財政規劃,2021-12-29 17:30:15,https://news.ltn.com.tw/news/politics/breaking...,國民黨黨主席朱立倫。（國民黨文傳會提供）〔記者楊媛婷／台北報導〕新竹市長林智堅日前宣布，為了...
...,...,...,...,...
534,說當立委時反萊牛 盧秀燕被議員質疑:明明有投贊成票,2021-12-07 10:13:24,https://news.ltn.com.tw/news/politics/breaking...,台中市長盧秀燕今天首度對四項公投表態，過去反美牛，現在反美豬。（記者唐在馨攝）〔記者唐在馨／...
535,反萊豬、福島食品 士電老總憂影響CPTPP,2021-12-07 05:30:00,https://news.ltn.com.tw/news/politics/paper/14...,士電總經理謝漢章。（記者林菁樺攝）〔記者林菁樺／台北報導〕士林電機總經理謝漢章昨參加士電變壓...
536,自由廣場》﹙北社評論﹚四個公投不同意（系列四之三） 公投綁大選，請投反對票,2021-12-07 05:30:00,https://talk.ltn.com.tw/article/paper/1488693,二○一八年，臺灣公投綁大選，一共有十個公投案，合併九合一選舉，還記得轟轟烈烈的到半夜還沒投完...
537,《觀察筆記》反馬之必要,2021-12-07 05:30:00,https://news.ltn.com.tw/news/politics/paper/14...,為反對而反對，馬英九。他的行政院在二○一二年三月五日深夜發布新聞，開放含「萊克多巴胺」美牛進...


In [78]:
ltn_politics.to_csv("ltnweb04.csv", index=False, encoding='UTF-8')

In [76]:
ltn_politics1 = pd.read_csv('./ltnweb01.csv', encoding = 'utf-8')
ltn_politics1['date'] = pd.to_datetime(ltn_politics1.date)
ltn_politics1

,title,date,link,content
0,連趕2場！苗縣宣講公投 賴清德：替年輕人鋪好邁向國際的道路,2021-12-07 17:34:00,https://news.ltn.com.tw/news/politics/breaking...,副總統賴清德宣講公投，表示替年輕人鋪好邁向國際社會的道路。（記者蔡政珉攝）〔記者蔡政珉／苗栗...
1,王美花：假如反美豬公投過了是搬石頭砸自己腳,2021-12-07 15:16:00,https://news.ltn.com.tw/news/politics/breaking...,「蔡英文&DPP」的Podcast頻道「耳朵出遊不出油」，邀請經濟部長王美花上節目受訪。（民...
2,自由開講》國人要全力動員，以「4不」護台保台,2021-12-07 11:45:00,https://talk.ltn.com.tw/article/breakingnews/3...,國民黨發言人江怡臻前幾天宣稱，林秉樞事件讓綠營產生破口，多份媒體民調指出，只要支持同意的選民...
3,誰在帶風向？議員逼四項公投表態和盧秀燕唇槍舌戰,2021-12-07 11:30:00,https://news.ltn.com.tw/news/politics/breaking...,民進黨議員蔡耀頡（右二）等人今天要求盧秀燕對四項公投表態，雙方數度唇槍舌戰，最後盧秀燕（右）...
4,自由開講》國共兩黨共同鎖定「反萊豬」操弄資訊！？,2021-12-07 11:15:00,https://talk.ltn.com.tw/article/breakingnews/3...,四項公投投票日將至，根據研究假訊息的IORG研究室公布最新報告指出，公投相關論述皆有中共發起...
...,...,...,...,...
2590,這是炒作打垮台灣豬 陳吉仲：國內絕不能使用萊克多巴胺,2020-09-02 09:27:00,https://news.ltn.com.tw/news/politics/breaking...,陳吉仲強調，國內絕對不能使用萊克多巴胺養豬。（記者朱沛雄攝）〔記者簡惠茹／台北報導〕因應美萊...
2591,確保豬農權益 陳吉仲：百億基金若不夠會再爭取百億,2020-09-01 13:53:00,https://news.ltn.com.tw/news/politics/breaking...,農委會主委陳吉仲今天到廣播電台說明美豬美牛政策。（資料照，記者簡惠茹攝）〔記者簡惠茹／台北報...
2592,美豬牛開放》嚴格把關食安 陳吉仲「5張圖表」說明,2020-08-29 11:07:00,https://news.ltn.com.tw/news/politics/breaking...,陳吉仲以圖表說明美豬進口與「台灣經貿發展」。（圖擷取自臉書）〔即時新聞／綜合報導〕台灣將開放...
2593,美豬牛開放》農委會：萊克多巴胺容許值 比照日韓訂嚴格標準,2020-08-28 16:19:00,https://news.ltn.com.tw/news/politics/breaking...,農委會強調，為確保國產豬肉的競爭力，國內養豬飼料用藥仍然不核准使用萊克多巴胺，台灣訂定的殘留...


In [77]:
ltn_politics = pd.concat([ltn_politics, ltn_politics1], ignore_index = True)
ltn_politics = ltn_politics.drop_duplicates(subset = ['link']).reset_index(drop = True)
ltn_politics = ltn_politics.sort_values('date', ascending = False).reset_index(drop = True)
ltn_politics

,title,date,link,content
0,總統今發表2022新年談話 「穩健執政 讓國家進步前進」,2022-01-01 05:30:00,https://news.ltn.com.tw/news/politics/paper/14...,蔡英文總統今天將在總統府大禮堂發表「二○二二新年談話」，向國人傳達今年的重要施政方針。（資料...
1,自由廣場》唉，這樣的楊志良……,2021-12-31 05:30:00,https://talk.ltn.com.tw/article/paper/1493110,曾任馬政府衛生署長的楊志良日前在一場群眾活動裡，除了用「假論文」的莫須有罪名扣在蔡英文總統身...
2,謝長廷請假缺席立院報告 陳以信不滿,2021-12-30 17:00:22,https://news.ltn.com.tw/news/politics/breaking...,立法院外交及國防委員會召委、國民黨立委陳以信批評謝長廷不願面對國會。（記者楊丞彧攝）〔記者楊...
3,朱酸：沒人在乎林智堅是否要選,2021-12-30 05:30:00,https://news.ltn.com.tw/news/politics/paper/14...,朱立倫昨反酸，沒人在乎林智堅是否要選大新竹。（資料照）〔記者楊媛婷／台北報導〕新竹市長林智堅...
4,酸沒人在乎林智堅選不選 朱立倫：歡迎蔡總統討論國土財政規劃,2021-12-29 17:30:15,https://news.ltn.com.tw/news/politics/breaking...,國民黨黨主席朱立倫。（國民黨文傳會提供）〔記者楊媛婷／台北報導〕新竹市長林智堅日前宣布，為了...
...,...,...,...,...
3118,這是炒作打垮台灣豬 陳吉仲：國內絕不能使用萊克多巴胺,2020-09-02 09:27:00,https://news.ltn.com.tw/news/politics/breaking...,陳吉仲強調，國內絕對不能使用萊克多巴胺養豬。（記者朱沛雄攝）〔記者簡惠茹／台北報導〕因應美萊...
3119,確保豬農權益 陳吉仲：百億基金若不夠會再爭取百億,2020-09-01 13:53:00,https://news.ltn.com.tw/news/politics/breaking...,農委會主委陳吉仲今天到廣播電台說明美豬美牛政策。（資料照，記者簡惠茹攝）〔記者簡惠茹／台北報...
3120,美豬牛開放》嚴格把關食安 陳吉仲「5張圖表」說明,2020-08-29 11:07:00,https://news.ltn.com.tw/news/politics/breaking...,陳吉仲以圖表說明美豬進口與「台灣經貿發展」。（圖擷取自臉書）〔即時新聞／綜合報導〕台灣將開放...
3121,美豬牛開放》農委會：萊克多巴胺容許值 比照日韓訂嚴格標準,2020-08-28 16:19:00,https://news.ltn.com.tw/news/politics/breaking...,農委會強調，為確保國產豬肉的競爭力，國內養豬飼料用藥仍然不核准使用萊克多巴胺，台灣訂定的殘留...
